# Homework 1: Information Retrieval
## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of three!
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

Since it is the first time we provide this homework please notify us if there is a bug/something is unclear, typo's exc..

5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [29]:
import numpy as np
import pandas as pd
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
import time
import os
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Maya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
tic = time.perf_counter() #Start timer

# Config

In [31]:
cwd=os.getcwd()
INPUT_FILE_PATH = os.path.join(cwd, "lyrics.csv")
BOW_PATH = os.path.join(cwd, "bow.csv")
N_ROWS = 100000
CHUNCK_SIZE = 5000
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1
COLS = [0]

## 1.1 Bag of words /TfIdf model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [32]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [33]:
def preprocess_sentence(sentence : str) -> List[str]:
    '''
    this function preprocesses a sentence by tokenizing it, removing stop words, stemming and removing punctuation and non-english words

    Parameters
    ----------
    sentence : str
        a string representing a sentence
    
    Returns
    -------
    List[str]
        a list of words
    '''
    output_sentence = []
    for word in word_tokenize(sentence):
        ### YOUR CODE HERE
        
        # Lower case the word
        lower_word = word.lower()
        
        # Ignores it if it's in the stopwords list
        if lower_word in stop_words:
            continue
        
        # Removes characters which are not in the allowed symbols
        # Create an empty string to store the output
        output_string = ""
        # Iterate over each character in the input string
        for letter in lower_word:
            # Check if the character is not equal to the specified letter
            if letter in allowed_symbols:
                # If it is not equal to the specified letter, add it to the output string
                output_string += letter

        # Stems it and appends it to the output sentence       
        stem_word = stemmer.stem(output_string)
        
        # Discards words with length <= 1
        if len(stem_word) <= 1:
            continue
        else:
            output_sentence.append(stem_word)
        
        ### END YOUR CODE
    return output_sentence

def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

In [34]:
class TfIdf:
    def __init__(self):
        #unigram_count is a dictionary that maps a word to the number of times it appears in the corpus
        self.unigram_count =  Counter()
        #bigram_count is a dictionary that maps a bigram to the number of times it appears in the corpus
        self.bigram_count = Counter()
        #document_term_frequency is a dictionary that maps a document id to the number of words in the document
        self.document_term_frequency = Counter()
        #word_document_frequency is a dictionary that maps a word to the number of documents it appears in
        self.word_document_frequency = {}
        #inverted_index is a dictionary that maps a word to a dictionary that maps a document id to the tf-idf weight of the word in the document
        self.inverted_index = {}
        #doc_norms is a dictionary that maps a document id to the norm of the document
        self.doc_norms = {}
        #n_docs is the number of documents in the corpus
        self.n_docs = -1
        #sentence_preprocesser is a function that preprocesses a sentence
        self.sentence_preprocesser = preprocess_sentence
        #bow_path is the path to the bag of words
        self.bow_path = BOW_PATH

    def update_counts_and_probabilities(self, sentence:List[str], document_id:int) -> None:
        '''
        this function updates the unigram and bigram counts and the inverted index given a sentence and a document id

        Parameters
        ----------
        sentence : List[str]
            a list of words
        document_id : int
            the id of the document
        
        Returns
        -------
        None
        '''
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        
        for i,word in enumerate(sentence):
            ### YOUR CODE HERE
            
            # Update self.unigram count (the amount of time each word is in the text)
            self.unigram_count[word] += 1
            
            # Update self.bigram count (two consecutive word occurances)
            if sentence_len > i + 1:  # if there is another word after the current one, update bigram count of the word
                self.bigram_count[sentence[i], sentence[i + 1]] += 1     
                            
            # Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}            
            count_dic = self.inverted_index.get(word, Counter())  # take the dictionary that holds for each files how many times the word appears 
            count_dic[document_id] += 1  # update the dictionary for the current file
            self.inverted_index[word] = count_dic  # update the word's dictionary
            
            ### END YOUR CODE
            

    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck: #sentence is a song (string)
                self.n_docs += 1 
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()
             
    def compute_word_document_frequency(self):
        '''
        this function computes the word document frequency for each word in the inverted index. e.g. if a word appears in 10 documents, the word document frequency is 10.
        '''
        for word in self.inverted_index.keys():
            ### YOUR CODE HERE
            self.word_document_frequency[word] = len(self.inverted_index[word].keys())
                            
            ### END YOUR CODE
    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        '''
        This function updates the inverted index with tf-idf weighting and computes the document norm for each document.
        '''
        for word in self.inverted_index.keys():  # go over the words in the inverted index dic
            df = self.word_document_frequency[word]  # Determine the count of files that contain the given word
            idf = np.log10((self.n_docs + 1) / float(df))  # calculate the Inverse Document Frequency
            word_quant = self.inverted_index[word]  # Retrieve the dictionary that indicates the frequency of the given word in each file
            for doc_id in word_quant.keys():  # go over the files that the word appears in
                tf = word_quant[doc_id] / self.document_term_frequency[doc_id]  # calculate tf 
                word_quant[doc_id] = tf * idf  # update the dictionary, wij instead of word count in the file
                self.doc_norms[doc_id] = self.doc_norms.get(doc_id, 0) + (word_quant[doc_id]) ** 2  # calculate file norm
            self.inverted_index[word] = word_quant  # update word's dic
                      
        ### END YOUR CODE
        for doc in self.doc_norms.keys():
            self.doc_norms[doc] = np.sqrt(self.doc_norms[doc]) 
            
    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)

In [35]:
tf_idf = TfIdf()
tf_idf.fit()

 95%|█████████▌| 20/21 [18:47<00:56, 56.40s/it]


## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.

In [36]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        #sentence_preprocesser is a function that preprocesses a sentence
        self.sentence_preprocesser = preprocess_sentence  
        #vocab is a set of all the words in the corpus
        self.vocab = set(tf_idf.unigram_count.keys())
        #n_docs is the number of documents in the corpus
        self.n_docs = tf_idf.n_docs
        #inverted_index is a dictionary that maps a word to a dictionary that maps a document id to the tf-idf weight of the word in the document
        self.inverted_index = tf_idf.inverted_index
        #word_document_frequency is a dictionary that maps a word to the number of documents it appears in
        self.word_document_frequency = tf_idf.word_document_frequency
        #doc_norms is a dictionary that maps a document id to the norm of the document
        self.doc_norms = tf_idf.doc_norms
        
    def rank(self, query:Dict[str,int], documents:Dict[str,Counter], metric:str) -> Dict[str, float]:
        '''
        this function ranks the documents according to the query and the chosen similarity metric

        Parameters
        ----------
        query : Dict[str,int]
            a dictionary of words and their counts in the query
        documents : Dict[str,Counter]
            a dictionary of documents that contain the query words and their tf-idf weights
        metric : str
            the similarity metric to use. can be 'cosine' or 'dot'
        
        Returns
        -------
        Dict[str, float]
            a dictionary of documents and their similarity to the query
        '''
        result = {} # key: DocID , value : float , simmilarity to query
        query_len = np.sum(np.array(list(query.values())))
        ### YOUR CODE HERE        
        for word in query.keys():  # go over the words in the query
            q_df = self.word_document_frequency[word]  # save the number of files that the word appears in them
            q_idf = np.log10((self.n_docs + 1)/float(q_df))  # calculate idf according to the formula
            q_tf = query[word]/query_len  # calculate tf according to the formula
            query[word] = q_tf * q_idf # calculate tfidf according to the formula
        # inner product - creating the numerator
        for term in documents.keys():  # go over the words in the query
            for doc_id in documents[term].keys():  # go over the docs the word appears in
                wij = documents[term][doc_id]  # take the tfidf value
                result[doc_id] = result.get(doc_id, 0) + wij * query[term]  # calculate the similarity of each doc to the query
                            
        ### END YOUR CODE
        if metric == 'cosine':
            ### YOUR CODE HERE
            q_norm = np.sqrt(np.sum(np.square(list(query.values())))) 
            for doc_id in result.keys():
                result[doc_id] = result[doc_id] / (q_norm * self.doc_norms[doc_id])  # normelaized the similarity

            ### END YOUR CODE
        return result
        
           
    def sort_and_retrieve_k_best(self, scores:Dict[str, float], k:int)-> List[str]:
        '''
        this function sorts the documents according to their similarity to the query and returns the k best documents
        '''
        ### YOUR CODE HERE 

        return sorted(scores, key=scores.get, reverse=True)[0:k]
    
        ### END YOUR CODE

    
    def reduce_query_to_counts(self, query:List)->  Counter:
        '''
        this function reduces the query to a dictionary of words and their counts
        '''
        ### YOUR CODE HERE
        return Counter(query)
        ### END YOUR CODE 
        
        
    def get_top_k_documents(self, query:str, metric:str , k=5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan 
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)
        
dr = DocumentRetriever(tf_idf)

In [37]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [38]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

[50221, 50233, 28634, 38867, 27766]
[26464, 38867, 7344, 43026, 43185]


In [39]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [0]).iloc[cosine_top_k]['Lyric']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
A Town Called Malice. Better stop dreaming of the quiet life. 'Cos it's the one we'll never know. And quit running for that runaway bus. 'Cos those rosey days are few. And stop apologising for the things you've never done,. 'Cos time is short and life is cruel. Well it's up to us to change. This town called malice uh uh yeah. Rows and rows of disused milk floats. stand dying in the dairy yard. And a hundred lonely housewives clutch empty milk. bottles to their hearts. Hanging out their old love letters on the line to dry. It's enough to make you stop believing when tears come. fast and furious. In a town called malice uh uh yeah. Struggle after struggle. Year after year. The atmosphere's a fine blend of ice. I'm almost stone cold dead. In a town called malice uh uh yeah. A whole street's belief in Sunday's roast beef. gets dashed against the Co-op. To either cut down on beer or the kids new gear. It's a big decision in a town 

# 1.3 term statistics:
Use "tf_idf" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [43]:
# 1. 
### YOUR SOLUTION HERE
u_w_amount = len(tf_idf.unigram_count.keys())
print(f'There are {u_w_amount} unique words')
### END YOUR SOLUTION

"""
### Your verbal solution here
There are 100220 unique words in the data set.
### End your verbal solution here
"""

# 2.
### YOUR SOLUTION HERE

potential_bigrams = u_w_amount*(u_w_amount-1)
bbigrams_amount = len(tf_idf.bigram_count.keys())
print(f'The potential bigram words amount is {potential_bigrams}')
print(f'There are actualy {bbigrams_amount} bigram words')

### END YOUR SOLUTION

"""
### Your verbal solution here

Out of all the possible combinations of words in the data, there are 10043948180 potential word bigrams. 
However, the actual number of bigrams we have is only 2484530. 
The reason for this difference is that the potential bigrams include all possible pairings of words, 
while the actual bigrams only consider pairs of words where one word comes before the other. 
Therefore, it's not surprising that the number of potential bigrams is greater than the number of actual bigrams.

### End your verbal solution here
"""

# 3.
### YOUR SOLUTION HERE

input_file_info = os.stat(INPUT_FILE_PATH)
print("Input file size is",input_file_info.st_size*0.000001,"MB")

output_file_info = os.stat(BOW_PATH)
print("Output file size is",output_file_info.st_size*0.000001,"MB")

### END YOUR SOLUTION

"""
### Your verbal solution here

The input file takes up 168.27 MB of storage space, whereas the output file only requires 76.09 MB. 
This difference in size is because the output file is based on a bag-of-words approach, 
which only includes the unique words from the inverted index. 
As a result, the output file requires less storage space compared to the input file.

### End your verbal solution here
"""

There are 100220 unique words
The potential bigram words amount is 10043948180
There are actualy 2484530 bigram words
Input file size is 168.274338 MB
Output file size is 76.095002 MB


'\n### Your verbal solution here\n\nThe input file takes up 168.27 MB of storage space, whereas the output file only requires 76.09 MB. \nThis difference in size is because the output file is based on a bag-of-words approach, \nwhich only includes the unique words from the inverted index. \nAs a result, the output file requires less storage space compared to the input file.\n\n### End your verbal solution here\n'

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (v) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [44]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
    
"""
for example - get_bigrams is a generator, which is an object we can loop on:
for ngram in get_bigrams(word):
    DO SOMETHING
"""

'\nfor example - get_bigrams is a generator, which is an object we can loop on:\nfor ngram in get_bigrams(word):\n    DO SOMETHING\n'

In [45]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        #unigram_counts is a dictionary that maps a word to its count in the corpus
        self.unigram_counts = unigram_counts
        #ngram_index is a dictionary that maps an ngram to a set of words that contain it
        self.ngram_index = {}
        #get_n_grams is a function that returns the ngrams of a word
        self.get_n_grams = get_n_gram
    
    def build_index(self) -> None:
        '''
        this function builds an index of ngrams to words that contain them
        the structure of the index is a dictionary of ngrams to sets of words
        '''
        ### YOUR CODE HERE
        for word in self.unigram_counts: # goes over the words
            for ngram in set(list(self.get_n_grams(word))):  # split the word into bigrams 
                if ngram in self.ngram_index:  # if the ngram is in the ngram index dictionary add the word to his list
                    self.ngram_index[ngram].append(word)
                else:  # if the ngram not in the ngram index dictionary add him for the first time 
                    self.ngram_index[ngram] = [word]
        return
        ### END YOUR CODE
        
    def get_top_k_words(self,word:str,k=5) -> (List[str], Dict[str, float]):
        '''
        this function builds an index of ngrams to words that contain them
        the structure of the index is a dictionary of ngrams to sets of words
        '''
        ### YOUR CODE HERE
        candidate_words = []  # a list to hold all the words that has mutual bigrams with the given word
        for bgram in self.get_n_grams(word):  # go over the bigrams of the word
            candidate_words += self.ngram_index[bgram]  # add all the words with mutual bigram to the candidate words
        # turn the list into a counter that counts the number of times each word appear with a mutual bigram
        
        word_counter = Counter(candidate_words)                
        jaccard_index =  {}  
        v_len = len(list(self.get_n_grams(word)))
        for word in word_counter:  # go over the counter dictionary
            w_len = len(list(self.get_n_grams(word)))  # number of bigrams in the current word
            jaccard_index[word] = (word_counter[word] / (v_len + w_len - word_counter[word]))  # calculate jaccard index
        top_k_words = sorted(jaccard_index, key=jaccard_index.get, reverse=True)[:k]  # sort by jaccard_index and get the first k similiar words
        return top_k_words, jaccard_index
        
        ### END YOUR CODE


class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)

Code Understanding Question:

In the "get top k words" function, does each candidate word have repeat ones or there is a channce that the same word will be iterated more than once? Explain your answer.

Answer here:

In [46]:
out_of_vocab_word = 'acress'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
candidate_words, scores = bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

In [47]:
candidate_words

['cress', 'ress', 'actress', 'cresson', 'recress']

In [48]:
for word in candidate_words:
    print(f'The jaccared score for the word {word} is {round(scores[word],2)}')

The jaccared score for the word cress is 0.8
The jaccared score for the word ress is 0.6
The jaccared score for the word actress is 0.57
The jaccared score for the word cresson is 0.57
The jaccared score for the word recress is 0.57


# The End - You did it!

In [49]:
#The time it took to run the entire code
toc = time.perf_counter()
print(f"The time it took to run the entire code is: {(toc - tic)/60} minuts")

The time it took to run the entire code is: 29.862189945000015 minuts
